In [ ]:
from ortools.sat.python import cp_model as cp

def readGrid(input_str):
    lines = input_str.strip().split('\n')
    dims = lines[0].strip().split()
    m, n = int(dims[0]), int(dims[1])
    raw_grid = [l.strip().split() for l in lines[1:] if l.strip()]
    return m, n, raw_grid

class MunraitoSolver:
    def __init__(self, rows, cols, grid_data):
        self.R = rows
        self.C = cols
        self.raw_grid = grid_data
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()

        # 决策变量
        self.stars = {}  # s[i,j]
        self.clouds = {} # c[i,j]
        
        # 辅助变量：记录光束状态
        # beam_lr[i, j]: 光从 (i,j) 向右射出
        # beam_rl[i, j]: 光从 (i,j) 向左射出
        # beam_ud[i, j]: 光从 (i,j) 向下射出
        # beam_du[i, j]: 光从 (i,j) 向上射出
        self.beam_lr = {} 
        self.beam_rl = {}
        self.beam_ud = {}
        self.beam_du = {}

        self._init_vars()

    def _init_vars(self):
        for r in range(self.R):
            for c in range(self.C):
                # 1. 基础状态变量
                self.stars[r, c] = self.model.NewBoolVar(f's_{r}_{c}')
                self.clouds[r, c] = self.model.NewBoolVar(f'c_{r}_{c}')
                
                # 2. 预处理 Planet 和 "-"
                cell_val = self.raw_grid[r][c]
                is_planet = cell_val != '-' and cell_val != 's' and cell_val != 'x' # 简单判断数字
                
                if is_planet:
                    # Planet 位置不能放星或云
                    self.model.Add(self.stars[r, c] == 0)
                    self.model.Add(self.clouds[r, c] == 0)
                else:
                    # 普通格子不能同时是星和云
                    self.model.Add(self.stars[r, c] + self.clouds[r, c] <= 1)
                
                # 3. 初始化光束变量
                self.beam_lr[r, c] = self.model.NewBoolVar(f'blr_{r}_{c}')
                self.beam_rl[r, c] = self.model.NewBoolVar(f'brl_{r}_{c}')
                self.beam_ud[r, c] = self.model.NewBoolVar(f'bud_{r}_{c}')
                self.beam_du[r, c] = self.model.NewBoolVar(f'bdu_{r}_{c}')

    def _add_beam_constraints(self, r, c, beam_out, beam_in_val):
        """
        通用函数：根据当前格子状态，决定光束如何传播。
        beam_out: 当前格子流出的光 (BoolVar)
        beam_in_val: 射入当前格子的光 (0, 1, or BoolVar)
        """
        cell_val = self.raw_grid[r][c]
        is_planet = cell_val != '-'
        
        s = self.stars[r, c]
        cl = self.clouds[r, c]

        if is_planet:
            # Planet 总是阻挡光线，输出为 0
            self.model.Add(beam_out == 0)
        else:
            # 逻辑：
            # 1. 如果是 Star -> 发光 (Out = 1)
            # 2. 如果是 Cloud -> 挡光 (Out = 0)
            # 3. 如果是 Empty -> 透光 (Out = In)
            
            # Case Star: Out = 1
            self.model.Add(beam_out == 1).OnlyEnforceIf(s)
            
            # Case Cloud: Out = 0
            self.model.Add(beam_out == 0).OnlyEnforceIf(cl)
            
            # Case Empty (Neither Star nor Cloud): Out = In
            # 注意：OnlyEnforceIf(s.Not(), cl.Not()) 表示当既非星也非云时生效
            self.model.Add(beam_out == beam_in_val).OnlyEnforceIf(s.Not(), cl.Not())

    def add_constraints(self):
        # 1. 标准数独类约束：每行每列正好 1 星 1 云
        for r in range(self.R):
            self.model.Add(sum(self.stars[r, c] for c in range(self.C)) == 1)
            self.model.Add(sum(self.clouds[r, c] for c in range(self.C)) == 1)
        
        for c in range(self.C):
            self.model.Add(sum(self.stars[r, c] for r in range(self.R)) == 1)
            self.model.Add(sum(self.clouds[r, c] for r in range(self.R)) == 1)

        # 2. 构建光束传播网络
        
        # (A) Left -> Right (判定 Planet 左侧是否亮)
        for r in range(self.R):
            for c in range(self.C):
                in_val = self.beam_lr[r, c-1] if c > 0 else 0
                self._add_beam_constraints(r, c, self.beam_lr[r, c], in_val)

        # (B) Right -> Left (判定 Planet 右侧是否亮)
        for r in range(self.R):
            for c in range(self.C - 1, -1, -1):
                in_val = self.beam_rl[r, c+1] if c < self.C - 1 else 0
                self._add_beam_constraints(r, c, self.beam_rl[r, c], in_val)

        # (C) Up -> Down (判定 Planet 上方是否亮)
        for c in range(self.C):
            for r in range(self.R):
                in_val = self.beam_ud[r-1, c] if r > 0 else 0
                self._add_beam_constraints(r, c, self.beam_ud[r, c], in_val)

        # (D) Down -> Up (判定 Planet 下方是否亮)
        for c in range(self.C):
            for r in range(self.R - 1, -1, -1):
                in_val = self.beam_du[r+1, c] if r < self.R - 1 else 0
                self._add_beam_constraints(r, c, self.beam_du[r, c], in_val)

        # 3. 匹配 Planet 数值约束
        for r in range(self.R):
            for c in range(self.C):
                val_str = self.raw_grid[r][c]
                if val_str != '-':
                    # 对 Planet 本身，检查其四周射入的光
                    # 注意：Planet 是障碍物，beam_XX[r,c] 输出必定为0，
                    # 所以我们检查的是 beam_XX[Neighbor]
                    target = int(val_str)
                    
                    # Bit 8: Up (Top side is lit) -> Comes from Up (beam_ud of row r-1)
                    is_up_lit = (target & 8) >> 3
                    beam_from_up = self.beam_ud[r-1, c] if r > 0 else 0
                    if is_up_lit:
                        self.model.Add(beam_from_up == 1)
                    else:
                        self.model.Add(beam_from_up == 0)

                    # Bit 4: Left (Left side is lit) -> Comes from Left (beam_lr of col c-1)
                    is_left_lit = (target & 4) >> 2
                    beam_from_left = self.beam_lr[r, c-1] if c > 0 else 0
                    if is_left_lit:
                        self.model.Add(beam_from_left == 1)
                    else:
                        self.model.Add(beam_from_left == 0)

                    # Bit 2: Down (Bottom side is lit) -> Comes from Down (beam_du of row r+1)
                    is_down_lit = (target & 2) >> 1
                    beam_from_down = self.beam_du[r+1, c] if r < self.R - 1 else 0
                    if is_down_lit:
                        self.model.Add(beam_from_down == 1)
                    else:
                        self.model.Add(beam_from_down == 0)
                        
                    # Bit 1: Right (Right side is lit) -> Comes from Right (beam_rl of col c+1)
                    is_right_lit = (target & 1)
                    beam_from_right = self.beam_rl[r, c+1] if c < self.C - 1 else 0
                    if is_right_lit:
                        self.model.Add(beam_from_right == 1)
                    else:
                        self.model.Add(beam_from_right == 0)

    def solve(self):
        status = self.solver.Solve(self.model)
        if status == cp.OPTIMAL or status == cp.FEASIBLE:
            print("Solution Found:")
            for r in range(self.R):
                row_out = []
                for c in range(self.C):
                    val_str = self.raw_grid[r][c]
                    if val_str != '-':
                        row_out.append(f" {val_str} ") # Keep planet
                    elif self.solver.Value(self.stars[r, c]) == 1:
                        row_out.append(" s ")
                    elif self.solver.Value(self.clouds[r, c]) == 1:
                        row_out.append(" x ")
                    else:
                        row_out.append(" - ")
                print("".join(row_out))
        else:
            print("No solution found.")

# --- 测试代码 ---
if __name__ == "__main__":
    # 你的示例输入
    input_data = """
6 6
3 - - - - 0
- - - - - -
- - 0 9 - -
- 6 - - - -
- - - - - 2
- - - - 1 -
"""
    # 注意：由于你给的示例在文本描述和grid矩阵中略有对其差异（最后一行是6行还是5行？），
    # 你的文字描述 grid 是 6行（含header5x5），但实际数据有6行。
    # 我根据txt通常格式假设第一行是 Dim，下面是Grid。
    # 你提供的Grid实际上最后一行有一个9。
    # 下面手动修正各种不对齐，构建一个最接近你描述的测试用例
    
    # 修正后的模拟输入数据 (5x5 矩阵)
    # 5 行 5 列
    # - - - - 3
    # - - - - -
    # - 6 - - -
    # - - - 1 -
    # - - 9 - -
    
    # 若你的示例其实是6行数据，请相应修改
    # 这里使用代码模拟读取
    
    raw_grid_clean = [
        "- - 4 - 2 - - 0 - - - 2".split(" "),
        "- 1 - - - - 4 - - - - -".split(" "),
        "- 0 0 0 - - - 0 - 1 - -".split(" "),
        "- - - - - - - - - - - 12".split(" "),
        "- 3 - - 12 0 - - - 2 - -".split(" "),
        "- - - - - - - - - - - -".split(" "),
        "- - - 3 - - - - 12 - - 0".split(" "),
        "- 8 - - - 9 - - - - - -".split(" "),
        "9 - - - 4 - - - - - - 0".split(" "),
        "- - 8 - - - - 2 - - - -".split(" "),
        "- 0 - 9 - - - - - - 12 -".split(" "),
        "0 - - - - 0 - - 1 - - -".split(" ")
    ]
    
    # 运行求解
    print("Solving Munraito (Sternenhaufen)...")
    solver = MunraitoSolver(12, 12, raw_grid_clean)
    solver.add_constraints()
    solver.solve()